In [1]:
import GetOldTweets3 as got
import numpy as np
import pandas as pd
import datetime
import time

In [2]:
df= pd.read_csv("data/2010/census_tweets_month_3.csv")
df.date.max()

'2010-03-31'

In [3]:
def get_month_tweets2(month,query,start_day=1,y=2020, hashtag=False):
    max_delta = 5
    
    date_date = datetime.date.today().replace(year=y).replace(day=start_day).replace(month=month)
    days_delta = max_delta
    next_date = date_date + datetime.timedelta(days=days_delta)
    

    next_month = datetime.date.today().replace(year=y).replace(day=28).replace(month=month)\
    + datetime.timedelta(days=4)
    until_date= next_month - datetime.timedelta(days=next_month.day)
    
    next_month = until_date + datetime.timedelta(days=1)

 
    if next_date.strftime("%Y-%m-%d") > next_month.strftime("%Y-%m-%d"):
        days_delta = (next_month - date_date).days
        print("hemano",days_delta)
        next_date = next_month
    
    today = datetime.date.today().strftime("%Y-%m-%d")
    
    if next_date.strftime("%Y-%m-%d") > today:
        days_delta = (datetime.date.today() - date_date).days
        print("hemano2",days_delta)
        next_date = datetime.date.today()
        
    while date_date.strftime("%Y-%m-%d") <= until_date.strftime("%Y-%m-%d") and date_date.strftime("%Y-%m-%d") < today:
        
        incomplete = True
        while incomplete:
            try:
                print("trying",date_date.strftime("%Y-%m-%d"),next_date.strftime("%Y-%m-%d"))
                tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                .setSince(date_date.strftime("%Y-%m-%d")).setUntil(next_date.strftime("%Y-%m-%d"))
                # Creation of list that contains all tweets
                tweets = got.manager.TweetManager.getTweets(tweetCriteria)
                # Creating list of chosen tweet data
                text_tweets = [[tweet.id,tweet.date,tweet.username, tweet.text,tweet.hashtags,tweet.geo,\
                                tweet.retweets,tweet.favorites,tweet.mentions,tweet.to] for tweet in tweets]
              
                if len(text_tweets) > 0 :
                    df = pd.DataFrame(text_tweets, columns=["id","timestamp","username","text","hashtags","geo","retweets","favorites","mentions","to"])
                    df["date"] = df.timestamp.dt.date
                    store_csv2(df,month,y,hashtag)

                print("New rows:",len(text_tweets))
                date_date = date_date + datetime.timedelta(days=days_delta)
                days_delta = min(max_delta, days_delta + 1)
                next_date = next_date + datetime.timedelta(days=days_delta)
                if next_date.strftime("%Y-%m-%d") > next_month.strftime("%Y-%m-%d"):
                    days_delta = (next_month - date_date).days
                    next_date = next_month
                if next_date.strftime("%Y-%m-%d") > today:
                    days_delta = (datetime.date.today() - date_date).days
                    next_date = datetime.date.today()
                
                
                incomplete = False
                
            except :
                print("Request error")
                days_delta = max(1,days_delta - 1)
                next_date = date_date + datetime.timedelta(days=days_delta)
                if next_date.strftime("%Y-%m-%d") > next_month.strftime("%Y-%m-%d"):
                    next_date = next_month
                if next_date.strftime("%Y-%m-%d") > today:
                    next_date = datetime.date.today()
                
                time.sleep(60)
    
  
    hasht=""
    if hashtag:
        hasht = "hashtag_"
        
    file_name = f'data/{y}/{hasht}census_tweets_month_{month}.csv'
    
   
    df = pd.read_csv(file_name)
    return df

def store_csv2(df,month,y=2020,hashtag=False):
    hasht=""
    if hashtag:
        hasht = "hashtag_"
        
    file_name = f'data/{y}/{hasht}census_tweets_month_{month}.csv'
    
    df.to_csv(file_name,mode = 'a', header= False, index = False)

In [4]:
def update_month(year,month,query):
    hashtag = "#" in query
    if hashtag:
        file_name = f"data/{year}/hashtag_census_tweets_month_{month}.csv"
    else:
        file_name = f"data/{year}/census_tweets_month_{month}.csv"
        

    file = pd.read_csv(file_name, parse_dates=["date"]).sort_values(by=["date"],ascending=False)
    if file.shape[0]==0:
        date = datetime.date.today().replace(day=1).replace(month=month).replace(year=year)
    else:
        date = file.date.dt.date.max() + datetime.timedelta(days=1)
    day = date.day
    
    if date.month==month:
        return get_month_tweets2(month, query, start_day=day,y=year, hashtag=hashtag )
    

In [6]:
q = "#census2020 OR #2020census"
df = update_month(2020,5,q)
df

hemano 2
trying 2020-05-30 2020-06-01
New rows: 517


,id,timestamp,username,text,hashtags,geo,retweets,favorites,mentions,to,date
0,1.256372e+18,2020-05-01 23:58:01+00:00,ForOurFutureNV,Did you complete your #2020Census yet? Did you...,#2020Census,NaN,7,9,NaN,NaN,2020-05-01
1,1.256372e+18,2020-05-01 23:57:47+00:00,robachma,#2020Census,#2020Census,NaN,0,1,NaN,GregMartinMusic,2020-05-01
2,1.256372e+18,2020-05-01 23:57:30+00:00,ISRAELPROPHET,Message: Why respond to the #2020Census? Censu...,#2020Census,NaN,0,0,NaN,NaN,2020-05-01
3,1.256372e+18,2020-05-01 23:56:54+00:00,ISRAELPROPHET1,Message: Why respond to the #2020Census? Censu...,#2020Census,NaN,1,1,NaN,NaN,2020-05-01
4,1.256372e+18,2020-05-01 23:54:55+00:00,CityFortBraggCA,Did you know the #2020census is required by th...,#2020census #censusfacts,NaN,0,2,NaN,NaN,2020-05-01
...,...,...,...,...,...,...,...,...,...,...,...
23327,1.266521e+18,2020-05-30 00:06:48+00:00,ColostrumSaves,Completing the #2020Census will have a big imp...,#2020Census,NaN,0,0,NaN,NaN,2020-05-30
23328,1.266521e+18,2020-05-30 00:05:28+00:00,CountTheNation,Congrats to our young digital activist for you...,#2020Census,NaN,4,6,NaN,NaN,2020-05-30
23329,1.266521e+18,2020-05-30 00:04:10+00:00,YAMAMerchants,Census result impact services like; Special ed...,#Yemeniscount #GetCountedNYC #CountMeIn #YAMAC...,NaN,0,1,NaN,NaN,2020-05-30
23330,1.266520e+18,2020-05-30 00:00:49+00:00,myBlackCounts,Shoutout to our partners @PANASanDiego For the...,#2020Census #AfroImmigrant #MyBlackCounts,NaN,1,2,@PANASanDiego,NaN,2020-05-30


In [24]:
df[df.date == df.date.max()]

,id,timestamp,username,text,hashtags,geo,retweets,favorites,mentions,to,date
16693,1.262533e+18,2020-05-19 00:00:00+00:00,stlouispark,Everyone counts in St. Louis Park! Help us be ...,#2020Census #Census #WeCountMN,NaN,0,1,@uscensusbureau,NaN,2020-05-19
16157,1.262827e+18,2020-05-19 19:26:55+00:00,arkansasccc,The latest #2020Census self-response rate for ...,#2020Census,NaN,3,1,NaN,NaN,2020-05-19
16179,1.262823e+18,2020-05-19 19:12:36+00:00,710KURV,Rio Grande Valley Ranks Low In 2020 Census Res...,#RGV #2020Census,NaN,0,0,NaN,NaN,2020-05-19
16178,1.262824e+18,2020-05-19 19:13:12+00:00,KansasSOS,There is still time to fill out the #2020Censu...,#2020Census #KansasCounts,NaN,20,20,NaN,NaN,2020-05-19
16177,1.262824e+18,2020-05-19 19:13:29+00:00,ReadyNationCA,The City of Riverside understands the importan...,#2020Census #workforce #economy #businesses #2...,NaN,0,1,@CSIUCR @CCAEJ,NaN,2020-05-19
...,...,...,...,...,...,...,...,...,...,...,...
16431,1.262771e+18,2020-05-19 15:44:20+00:00,ISRAELPROPHET,"Once operations for the #2020Census resume, we...",#2020Census #AmericaCounts,NaN,0,0,NaN,NaN,2020-05-19
16430,1.262771e+18,2020-05-19 15:45:06+00:00,PhiladelphiaGov,"Don’t make a $21,000 mistake. That’s how much ...",#2020Census,NaN,25,33,NaN,NaN,2020-05-19
16447,1.262765e+18,2020-05-19 15:20:55+00:00,DorchesterSCGov,Filling out the #Census makes sure federal dol...,#Census #ShapeYourFuture #2020census #MakeItCo...,NaN,3,2,NaN,NaN,2020-05-19
16429,1.262771e+18,2020-05-19 15:45:12+00:00,FreeLibrary,Even though our minds are on the pandemic and ...,#Census2020 #PhillyCounts,NaN,1,2,NaN,NaN,2020-05-19


In [4]:
text_query = "census"
# text_query = "#2020census OR #census2020"
df = update_month(2020,4,text_query)
df

An error occured during an HTTP request: HTTP Error 503: Service Temporarily Unavailable
Try to open in browser: https://twitter.com/search?q=census%20since%3A2020-04-10%20until%3A2020-04-11&src=typd
Request error
2020-04-10 3514
2020-04-11 3055
An error occured during an HTTP request: HTTP Error 503: Service Temporarily Unavailable
Try to open in browser: https://twitter.com/search?q=census%20since%3A2020-04-12%20until%3A2020-04-13&src=typd
Request error
An error occured during an HTTP request: HTTP Error 503: Service Temporarily Unavailable
Try to open in browser: https://twitter.com/search?q=census%20since%3A2020-04-12%20until%3A2020-04-13&src=typd
Request error
An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=census%20since%3A2020-04-12%20until%3A2020-04-13&src=typd
Request error
An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?

,id,timestamp,username,text,hashtags,geo,retweets,favorites,mentions,to,date
0,1245863530223374336,2020-04-02 23:59:47+00:00,evduzit,Oh it’s Census time??,NaN,NaN,0,1,NaN,NaN,2020-04-02
1,1245863445259415558,2020-04-02 23:59:27+00:00,Ujamidaize,No ooo... I just came to do census for Angel Gabe,NaN,NaN,0,0,NaN,ETUKMMA,2020-04-02
2,1245863357149614080,2020-04-02 23:59:06+00:00,tonetalks,“African Americans account for 35% of cases & ...,NaN,NaN,19,30,@MehrsaBaradaran,kat__stafford,2020-04-02
3,1245863341693648897,2020-04-02 23:59:02+00:00,HermistonHerald,You know what's a great quarantine activity? F...,NaN,NaN,0,0,NaN,NaN,2020-04-02
4,1245863156879986689,2020-04-02 23:58:18+00:00,Zach_Case_,I’m 28 years old and just completed my first c...,NaN,NaN,0,3,NaN,NaN,2020-04-02
...,...,...,...,...,...,...,...,...,...,...,...
105862,1255648241309421568,2020-04-30 00:00:44+00:00,linc_nyc,#2020Census is a chance to shape your communit...,#2020Census #GetCountedNYC #2020Census #LetALL...,NaN,3,3,NaN,NaN,2020-04-30
105863,1255648185604857856,2020-04-30 00:00:30+00:00,AR_CAA,AR is only just shy of hitting the 50% census ...,NaN,NaN,1,2,NaN,NaN,2020-04-30
105864,1255648099969662988,2020-04-30 00:00:10+00:00,CalEndow,African Americans have historically been under...,#Census2020 #EveryoneCounts,NaN,3,5,NaN,NaN,2020-04-30
105865,1255648093992898561,2020-04-30 00:00:09+00:00,RockTheVote,Black millennials and generation Z---you have ...,#CountMeBlack #CountBlackYouth,NaN,6,13,NaN,NaN,2020-04-30
